In [1]:

# from src.VLLMServer import launch_command,get_client_dict,ask_llm,ask_llm_prompt
from tqdm import tqdm
from datasets import load_dataset
from src.code_utils import extract_code_block
from src.process_openmath_ja import parse_record, eval_answer
from vllm import SamplingParams, LLM
import json
import os


os.environ["CUDA_VISIBLE_DEVICES"]="1"
# %%
out_path = "processed/data_test.jsonl"

# %%

/home/setup/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-20 15:25:25,491	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:


# load dataset
ds = load_dataset("kunishou/OpenMathInstruct-1-1.8m-ja",
                  split="train", streaming=False)

ds = ds.shuffle()

# %%
model_name = "microsoft/Phi-3-medium-128k-instruct"
model_name="cyberagent/calm3-22b-chat"

llm = LLM(model=model_name, trust_remote_code=True,
          max_model_len=8000
          )

INFO 07-20 15:25:44 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='cyberagent/calm3-22b-chat', speculative_config=None, tokenizer='cyberagent/calm3-22b-chat', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=cyberagent/calm3-22b-chat)


/home/setup/miniconda3/envs/llmeval/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 07-20 15:25:44 utils.py:660] Found nccl from library /home/setup/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 07-20 15:25:45 selector.py:27] Using FlashAttention-2 backend.
INFO 07-20 15:25:46 weight_utils.py:199] Using model weights format ['*.safetensors']


In [ ]:


# %%
pre_command = """次の問題を解きなさい｡出力事項は以下のとおりである｡
#解説: 問題を解くための基本的な考え方を日本語で出力する｡
#Python: 計算プログラム｡以下の例を参考に､出力形式を守ること｡
```
#a+bの計算を行う
a=1
b=2
print(a+b)
```
#問題
"""

# %%
batch_size = 30
idx = 0


for j in tqdm(range(int(len(ds)/batch_size))):

    prompts = []
    answer_list = []
    question_list = []
    idx_list = []

    for i in range(batch_size):
        record = ds[idx]
        idx += 1
        # 問題文と回答の取得
        question, answer = parse_record(record)
        answer_list.append(answer)
        question_list.append(question)
        problem_id = record["index"]
        idx_list.append(idx)

        # promptの生成
        ja_text = pre_command+question
        prompt = f"""<|user|>
{ja_text}<|end|>
<|assistant|>"""
        prompts.append(prompt)

    # 回答の生成
    outputs = llm.generate(
        prompts,
        sampling_params=SamplingParams(
            temperature=0.1,
            max_tokens=1024,
            repetition_penalty=1.1,
        )
    )

    # 回答のチェック
    for i in range(len(outputs)):
        res = outputs[i].outputs[0].text.strip()
        res = res.replace("\'", "`")

        # 生成されたコードを実際にまわして評価する
        explanation_block = res[:res.find("```")]
        try:
            code_block = extract_code_block(res)
        except:
            continue

        answer = answer_list[i]
        question = question_list[i]
        # 真偽判定
        is_correct_answer = eval_answer(code_block, answer, verbose=False)

        generated_prompt = explanation_block+"\n"+code_block
        if is_correct_answer:
            new_record = {
                "id": idx_list[i],
                "question": question,
                "prediction": generated_prompt,
                "answer": answer,

            }
            with open(out_path, "a") as f:
                f.write(json.dumps(new_record, ensure_ascii=False)+"\n")

# %%